In [ ]:
import os
import sys
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc, recall_score
from sklearn.model_selection import train_test_split



#Set random seeds
torch.manual_seed(102)
np.random.seed(102)

#Define CNN configuration
CFG = {
    'cfg04': [8, 'M', 16, 'M'],  # CNN config
}

#Define global variables
MAX_AA_LEN = 24  #Maximum sequence length
K = 100          # Top k-mer
PC = 15          #PCA feature dimension

#Load amino acid PCA features
def load_aa_pc():
    aa_pc_map = {}
    file_path = "../Data/PCA15.txt"
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines[1:]:
            items = line.strip().split('\t')
            aa_name = items[0]
            aa_pca = [float(x) for x in items[1:]]
            aa_pc_map[aa_name.upper()] = aa_pca[:PC]  
    return aa_pc_map

#Global loading of PCA features once
aa_pc_map = load_aa_pc()

#Obtain PCA features of AAindex
def get_aaidx_feature(seq, aa_pc_map):
    features = []
    for aa in seq:
        features.append(aa_pc_map.get(aa.upper(), [0] * PC))
    return features

#Generate sequence input
def generate_input(sps, sp_lbs, ins_num, max_len, aa_pc_map):
    xs_seq = np.zeros((len(sps), ins_num, max_len, PC), dtype=np.float32)
    ys = np.zeros(len(sps), dtype=np.float32)
    for i, (sp, label) in enumerate(zip(sps, sp_lbs)):
        ys[i] = label
        for j, tcr in enumerate(sp[:ins_num]):
            tcr_seq = tcr[0][:max_len]
            features = get_aaidx_feature(tcr_seq, aa_pc_map)
            if len(features) < max_len:
                features += [[0] * PC] * (max_len - len(features))
            xs_seq[i, j] = np.array(features, dtype=np.float32)
    return xs_seq, ys

#Generate k-mer abundance features
def generate_kmer_input(sps, sp_lbs, ins_num):
    xs_kmer = np.zeros((len(sps), ins_num), dtype=np.float32)
    ys = np.zeros(len(sps), dtype=np.float32)
    for i, (sp, label) in enumerate(zip(sps, sp_lbs)):
        ys[i] = label
        for j, tcr in enumerate(sp[:ins_num]):
            xs_kmer[i, j] = float(tcr[1])
    return xs_kmer, ys

#Load data
def load_data(sample_dir):
    training_data, training_labels = [], []
    for file in os.listdir(sample_dir):
        if file.endswith('.txt') or file.endswith('.tsv'):
            sample_data = read_tsv(os.path.join(sample_dir, file), [0, 1], skip_1st=True)
            if len(sample_data) > 0:
                training_data.append(sample_data)
                if "P" in file:
                    training_labels.append(1)
                elif "H" in file:
                    training_labels.append(0)
                else:
                    print(f"Unknown label type: {file}")
            else:
                print(f"File {file} is empty, skipped")
    return training_data, training_labels

#Read TSV file
def read_tsv(filename, inf_ind, skip_1st=False):
    data = []
    with open(filename, 'r') as f:
        if skip_1st:
            next(f)
        for line in f:
            if line.strip() == '':
                continue
            parts = line.strip().split('\t')
            if len(parts) >= max(inf_ind) + 1:
                data.append([parts[i] for i in inf_ind])
            else:
                print(f"Incomplete lines exist in file {filename}, skipped")
    return data

#Sequence model
class B_Seq_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_dim = PC  #Feature dimension

        self.conv1d_3 = self.Conv1d(CFG['cfg04'], kernel_size=3)
        self.conv1d_4 = self.Conv1d(CFG['cfg04'], kernel_size=4)
        self.conv1d_5 = self.Conv1d(CFG['cfg04'], kernel_size=5)

        self.rnnlayer_b = nn.LSTM(input_size=self.feature_dim, hidden_size=64, num_layers=2, batch_first=True)
        self.rnn_dense = nn.Linear(64, 32)

        self.dropout = nn.Dropout(p=0.5)
        self.FClayer_1 = None  
        self.FClayer_2 = nn.Linear(512, 64)
        self.output_layer = nn.Linear(64, 32)

    def forward(self, x_b):
        
        batch_size, ins_num, seq_len, feature_dim = x_b.size()
        x_b = x_b.view(-1, seq_len, feature_dim)  


        x_b_cnn = x_b.permute(0, 2, 1)  

        cnn_b_3 = self.conv1d_3(x_b_cnn)
        cnn_b_4 = self.conv1d_4(x_b_cnn)
        cnn_b_5 = self.conv1d_5(x_b_cnn)

 
        cnn_b_3 = cnn_b_3.view(cnn_b_3.size(0), -1)
        cnn_b_4 = cnn_b_4.view(cnn_b_4.size(0), -1)
        cnn_b_5 = cnn_b_5.view(cnn_b_5.size(0), -1)


        rnn_out, _ = self.rnnlayer_b(x_b)  
        rnn_out = self.rnn_dense(rnn_out[:, -1, :]) 


        x = torch.cat([cnn_b_3, cnn_b_4, cnn_b_5, rnn_out], dim=1)

    
        if self.FClayer_1 is None:
            self.FClayer_1 = nn.Linear(x.size(1), 512).to(x.device)


        x = self.dropout(torch.relu(self.FClayer_1(x)))
        x = self.dropout(torch.relu(self.FClayer_2(x)))
        x = self.output_layer(x)  # [batch_size * ins_num, 32]


        x = x.view(batch_size, ins_num, -1).mean(dim=1)  # [batch_size, 32]
        return x

    def Conv1d(self, cfg, kernel_size):
        layers = []
        in_channels = self.feature_dim
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool1d(kernel_size=2, stride=2)]
            else:
                layers += [
                    nn.Conv1d(in_channels, v, kernel_size=kernel_size, padding=kernel_size // 2),
                    nn.ReLU(inplace=True)
                ]
                in_channels = v
        return nn.Sequential(*layers)

#Abundance model
class B_K_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(100, 512)
        self.dense_layer1 = nn.Linear(512, 64)
        self.output_layer = nn.Linear(64, 32)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        x = self.dropout(x)
        x = torch.relu(self.dense_layer1(x))
        x = self.dropout(x)
        x = self.output_layer(x)
        return x

#Integrated model
class B_EnsembleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, seq_out, kmer_out):
        x = torch.cat([seq_out, kmer_out], dim=1)
        x = torch.relu(self.input_layer(x))
        x = self.dropout(x)
        x = self.output_layer(x)
        return x

#Define evaluation indicators
def metrics(all_preds, all_labels, threshold=0.5):
    all_probs = torch.sigmoid(torch.tensor(all_preds)).numpy()
    binary_preds = (all_probs >= threshold).astype(int)
    conf_matrix = confusion_matrix(all_labels, binary_preds)
    accuracy = accuracy_score(all_labels, binary_preds)
    sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1]) if (conf_matrix[1, 0] + conf_matrix[1, 1]) != 0 else 0
    specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1]) if (conf_matrix[0, 0] + conf_matrix[0, 1]) != 0 else 0
    auc_score = roc_auc_score(all_labels, all_probs)
    return accuracy, sensitivity, specificity, auc_score

#Define evaluation function
def evaluate_combined(seq_model, kmer_model, ensemble_model, criterion, data_loader, device="cuda"):
    seq_model.eval()
    kmer_model.eval()
    ensemble_model.eval()

    total_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for seq_input, kmer_input, labels in data_loader:
            seq_input = seq_input.to(device)
            kmer_input = kmer_input.to(device)
            labels = labels.to(device).view(-1, 1)

            # Forward pass
            seq_output = seq_model(seq_input)
            kmer_output = kmer_model(kmer_input)
            final_output = ensemble_model(seq_output, kmer_output)

            # Compute loss
            loss = criterion(final_output, labels)
            total_loss += loss.item()

            all_preds.extend(final_output.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    return avg_loss, all_preds, all_labels


#Define training function
from tqdm import tqdm
def train_combined(fold, seq_model, kmer_model, ensemble_model, criterion, optimizer, train_loader, valid_loader, epoches=100, device="cuda"):
    epoch_train_losses = []
    epoch_valid_losses = []

    #Introduce early stop mechanism
    model_path = f'../Model/{Cancer_name}_fold{fold}_best.pt'
    early_stopping = EarlyStopping(PATIENCE, path=model_path, verbose=False)

    with tqdm(total=epoches) as t:
        t.set_description(f'{Cancer_name} - Fold {fold}')
        for epoch in range(epoches):
            seq_model.train()
            kmer_model.train()
            ensemble_model.train()
            total_loss = 0.0

            for seq_input, kmer_input, labels in train_loader:
                seq_input = seq_input.to(device)
                kmer_input = kmer_input.to(device)
                labels = labels.to(device).view(-1, 1)

                # Forward pass
                seq_output = seq_model(seq_input)
                kmer_output = kmer_model(kmer_input)
                final_output = ensemble_model(seq_output, kmer_output)

                # Compute loss
                loss = criterion(final_output, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            avg_train_loss = total_loss / len(train_loader)
            epoch_train_losses.append(avg_train_loss)


            valid_loss, _, _ = evaluate_combined(seq_model, kmer_model, ensemble_model, criterion, valid_loader, device=device)
            epoch_valid_losses.append(valid_loss)

            t.set_postfix(loss=avg_train_loss, valid_loss=valid_loss)
            t.update(1)

     
            early_stopping(valid_loss, seq_model, kmer_model, ensemble_model)
            if early_stopping.early_stop:
                print("Early stop trigger")
                break

    
    seq_model.load_state_dict(torch.load(model_path)['seq_model'])
    kmer_model.load_state_dict(torch.load(model_path)['kmer_model'])
    ensemble_model.load_state_dict(torch.load(model_path)['ensemble_model'])

#Introduce early stop mechanism
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, seq_model, kmer_model, ensemble_model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, seq_model, kmer_model, ensemble_model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'Early stop counter: {self.counter} / {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, seq_model, kmer_model, ensemble_model)
            self.counter = 0

    def save_checkpoint(self, val_loss, seq_model, kmer_model, ensemble_model):
        if self.verbose:
            print(f'Verify loss reduction ({self.val_loss_min:.6f} --> {val_loss:.6f})。Save Model...')
        torch.save({
            'seq_model': seq_model.state_dict(),
            'kmer_model': kmer_model.state_dict(),
            'ensemble_model': ensemble_model.state_dict()
        }, self.path)
        self.val_loss_min = val_loss


In [ ]:
# Define sigmoid function              
def sigmoid(x):
    return 1 / (1 + np.exp(-x))    
    
# 5-fold cross-validation
k_fold = 5
kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)

BATCH_SIZE = 64   
NUM_EPOCHES = 2000  
PATIENCE = 300  

all_accuracies = []  
all_sensitivities = [] 
all_specificities = [] 
all_aucs = []  

device = "cuda"
# Cancers name
Cancer_list = ["BC","BL","UBC","BRCA","CC","CRC","DLBCL","GBM","HCC","Lung ADC","KS","Lung","MCC","Melanoma","MF","NSCLC","OS","OC","PDAC","PCa","RCC","SCLC"]
results = []
results_ROC = []

for Cancer_name in Cancer_list:
    data_dir = f'../Data/{Cancer_name}'
    training_data, training_labels = load_data(data_dir)
    print(f"Working on {Cancer_name} dataset: {len(training_data)} samples")


    all_preds = []
    all_labels = []


    for fold, (train_idx, test_idx) in enumerate(kf.split(training_data)):
        train_data = [training_data[i] for i in train_idx]
        train_labels = [training_labels[i] for i in train_idx]
        test_data = [training_data[i] for i in test_idx]
        test_labels = [training_labels[i] for i in test_idx]

        
        train_data, valid_data, train_labels, valid_labels = train_test_split(
            train_data, train_labels, test_size=0.2, random_state=1234
        )

        
        train_seq_input, train_seq_labels = generate_input(train_data, train_labels, K, MAX_AA_LEN, aa_pc_map)
        valid_seq_input, valid_seq_labels = generate_input(valid_data, valid_labels, K, MAX_AA_LEN, aa_pc_map)
        test_seq_input, test_seq_labels = generate_input(test_data, test_labels, K, MAX_AA_LEN, aa_pc_map)

        train_kmer_input, _ = generate_kmer_input(train_data, train_labels, K)
        valid_kmer_input, _ = generate_kmer_input(valid_data, valid_labels, K)
        test_kmer_input, _ = generate_kmer_input(test_data, test_labels, K)

        train_dataset = Data.TensorDataset(
            torch.Tensor(train_seq_input), torch.Tensor(train_kmer_input), torch.Tensor(train_seq_labels)
        )
        valid_dataset = Data.TensorDataset(
            torch.Tensor(valid_seq_input), torch.Tensor(valid_kmer_input), torch.Tensor(valid_seq_labels)
        )
        test_dataset = Data.TensorDataset(
            torch.Tensor(test_seq_input), torch.Tensor(test_kmer_input), torch.Tensor(test_seq_labels)
        )

        train_loader = Data.DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
        valid_loader = Data.DataLoader(valid_dataset, BATCH_SIZE, shuffle=False)
        test_loader = Data.DataLoader(test_dataset, BATCH_SIZE, shuffle=False)


        
        seq_model = B_Seq_model().to(device)
        kmer_model = B_K_model().to(device)
        ensemble_model = B_EnsembleModel().to(device)


        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(
            list(seq_model.parameters()) +
            list(kmer_model.parameters()) +
            list(ensemble_model.parameters()),
            lr=0.001
        )


        train_combined(fold, seq_model, kmer_model, ensemble_model, criterion, optimizer, train_loader, valid_loader, epoches=NUM_EPOCHES, device=device)


        test_loss, preds, labels = evaluate_combined(seq_model, kmer_model, ensemble_model, criterion, test_loader, device=device)
        all_preds.extend(preds)
        all_labels.extend(labels)


    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    accuracy, sensitivity, specificity, auc = metrics(all_preds, all_labels)
    print(f"Mean Accuracy ({Cancer_name}): {accuracy:.4f}")
    print(f"Mean Sensitivity ({Cancer_name}): {sensitivity:.4f}")
    print(f"Mean Specificity ({Cancer_name}): {specificity:.4f}")
    print(f"Mean AUC ({Cancer_name}): {auc:.4f}")

    results.append({
        'Cancer': Cancer_name,
        'accuracy': accuracy,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'auc': auc
    })

    results_ROC.append({
        'Cancer': Cancer_name,
        'auc': auc,
        'all_preds': all_preds,
        'all_labels': all_labels
    })
